In [20]:
from pathlib import Path
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import imageio

def find_matching_files(directory, pattern):
    directory_path = Path(directory)
    filelist = directory_path.glob(pattern)
    return list(filelist)

# EXP - BIOGEM/ECOGEM - VARIABLES

directory_path = "../model"
file_pattern = "muffin.u067bc.PO4.8P8Z.EXP3*/biogem/fields_biogem_2d.nc"
var = 'eco2D_Plankton_C_Total'

## predefined dictionary of variables
var_dict = {'eco2D_Uptake_Fluxes_C': 'Net Primary Production (mmol C m$^{-3}$ d$^{-1}$)',
            'eco2D_Diversity_Simpson': 'Diversity (Simpson)',
            'eco2D_Plankton_C_Total': 'Total Carbon (mmol C m$^{-3}$)',
            'eco2D_Size_Mean': 'Mean Size (mm)',
            'ocn_sur_temp': 'Surface Temperature ($^{\circ}$C)',
            'ocn_sur_PO4': 'Surface PO$_4$ (mol kg$^{-1}$)',
            'ocn_sur_pH': 'Surface pH',
            'phys_w': 'Vertical Velocity (m s$^{-1}$)',
            'carb_sur_ohm_cal': 'Surface Calcite Saturation State',
            }

l = find_matching_files(directory_path, file_pattern)
vars = [xr.open_dataset(f)[var] for f in l]
mean_var = sum(vars) / len(vars)

KeyError: 'eco2D_Plankton_C_Total'

In [ ]:
# Create figure
fig, ax = plt.subplots()

# Add title and labels
ax.set_xlabel('Time')
ax.set_ylabel(var_dict[var])

## auto adjust vmin and vmax
vmin = mean_var.min().values
vmax = mean_var.max().values

# Initialize imshow
im = ax.imshow(mean_var.isel(time=0).values[::-1], vmin=vmin, vmax=vmax, animated=True)
cbar = fig.colorbar(im)

# Function to update plot
def update(i):

    # Update data
    im.set_array(mean_var.isel(time=i).values[::-1])

    # Update title with timestamp
    fig.suptitle(f'Time: {mean_var.time[i].values}', fontsize=14)

    # Draw updated plot
    fig.canvas.draw()
    return [im]

ani = animation.FuncAnimation(fig, update, frames=len(mean_var.time))

ani.save('../output/animation.mp4', writer='ffmpeg')

plt.show()